## Day 46 Lecture 1 Assignment

In this assignment, we will calculate a distance matrix for geographical Starbucks data and use it to identify locations that are close together and far apart. We will perform clustering on this dataset later on.

We will be using the "haversine" package to compute geographical distance. It can be pip installed.

In [1]:
#!pip install haversine
# installing loacally with conda install haversine

In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from haversine import haversine

Below is a convenience function for calculating geographical distance matrices using lat-long data.

In [3]:
def geo_sim_matrix(df, col_name = 'Coordinates'):
    """
    A function that computes a geographical distance matrix (in miles).
    Each row in the dataframe should correspond to one location.
    In addition, the dataframe must have a column containing the lat-long of each location as a tuple (i.e. (lat, long)).
    
    Parameters:
        df (pandas dataframe): an nxm dataframe containing the locations to compute similarities between.
        col (string): the name of the column containing the lat-long tuples.
        
    Returns:
        distance (pandas dataframe): an nxn distance matrix between the geographical coordinates of each location.
    """
    
    df = df.copy()
    df.reset_index(inplace=True)
    haver_vec = np.vectorize(haversine, otypes=[np.float32])
    distance = df.groupby('index').apply(lambda x: pd.Series(haver_vec(df[col_name], x[col_name])))
    distance = distance / 1.609344  # converts to miles
    distance.columns = distance.index
    
    return distance


This dataset contains the latitude and longitude (as well as several other details we will not be using) of every Starbucks in the world as of February 2017. Each row consists of the following features, which are generally self-explanatory:

- Brand
- Store Number
- Store Name
- Ownership Type
- Street Address
- City
- State/Province
- Country
- Postcode
- Phone Number
- Timezone
- Longitude
- Latitude

Load in the dataset.

In [4]:
# answer goes here
starbucks_df = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/starbucks_locations.csv')

Begin by narrowing down the dataset to a specific geographic area of interest. Since we will need to manually compute a distance matrix, which will be on the order of $n^{2}$ in terms of size, we would recommend choosing an area with 3000 or less locations. In this example, we will use Hawaii, which has about 100 locations; for reference, California has about 2800 locations. Feel free to choose a different region that is of more interest to you, if desired.

Subset the dataframe to only include records for Starbucks locations in Hawaii.

In [5]:
# answer goes here
starbucks_df.head()




,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude
0,Starbucks,47370-257954,"Meritxell, 96",Licensed,"Av. Meritxell, 96",Andorra la Vella,7,AD,AD500,376818720,GMT+1:00 Europe/Andorra,1.53,42.51
1,Starbucks,22331-212325,Ajman Drive Thru,Licensed,"1 Street 69, Al Jarf",Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.42
2,Starbucks,47089-256771,Dana Mall,Licensed,Sheikh Khalifa Bin Zayed St.,Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.39
3,Starbucks,22126-218024,Twofour 54,Licensed,Al Salam Street,Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.38,24.48
4,Starbucks,17127-178586,Al Ain Tower,Licensed,"Khaldiya Area, Abu Dhabi Island",Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.54,24.51


In [6]:
starbucks_hi_df = starbucks_df.loc[starbucks_df['State/Province'] == 'HI'].copy()
print(starbucks_hi_df.shape)
starbucks_hi_df.head()

(99, 13)


,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude
17202,Starbucks,21034-73360,Aiea Shopping Center,Company Owned,99-115 Aiea Heights Drive #125,Aiea,HI,US,967013913,808-484-1488,GMT-10:00 Pacific/Honolulu,-157.93,21.38
17203,Starbucks,21053-99755,Stadium Marketplace,Company Owned,4561 Salt Lake Boulevard,Aiea,HI,US,968183167,808-488-9313,GMT-10:00 Pacific/Honolulu,-157.93,21.37
17204,Starbucks,21006-10033,Kaonohi St & Kam Hwy - Pearlridge,Company Owned,98-125 Kaonohi Street,Aiea,HI,US,967012318,808-484-9548,GMT-10:00 Pacific/Honolulu,-157.94,21.38
17205,Starbucks,21005-10034,Pearlridge Mall Uptown,Company Owned,98-1005 Moanalua Road,Aiea,HI,US,967014705,808-484-9355,GMT-10:00 Pacific/Honolulu,-157.94,21.39
17206,Starbucks,21063-101700,Waimalu Shopping Center,Company Owned,"98-1277 Kaahumanu Street, Building E, Unit 7, ...",Aiea,HI,US,967015314,808-484-5802,GMT-10:00 Pacific/Honolulu,-157.95,21.39


The haversine package takes tuples with 2 numeric elements and interprets them as lat-long to calculate distance, so add a new column called "Coordinates" that converts the lat and long in each row into a tuple. In other words, the last two columns of the dataframe should initially look like this:

**Latitude, Longitude**  
39.14, -121.64  
34.13, -116.40  
...

After adding the new column, the last three columns should look like this:

**Latitude, Longitude, Coordinates**  
39.14, -121.64,   (39.14, -121.64)  
34.13, -116.40,   (34.13, -116.40)  
...

In [7]:
# answer goes here
starbucks_hi_df['Coordinates'] = list(zip(starbucks_hi_df['Latitude'], starbucks_hi_df['Longitude']))
starbucks_hi_df.head()

,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude,Coordinates
17202,Starbucks,21034-73360,Aiea Shopping Center,Company Owned,99-115 Aiea Heights Drive #125,Aiea,HI,US,967013913,808-484-1488,GMT-10:00 Pacific/Honolulu,-157.93,21.38,"(21.38, -157.93)"
17203,Starbucks,21053-99755,Stadium Marketplace,Company Owned,4561 Salt Lake Boulevard,Aiea,HI,US,968183167,808-488-9313,GMT-10:00 Pacific/Honolulu,-157.93,21.37,"(21.37, -157.93)"
17204,Starbucks,21006-10033,Kaonohi St & Kam Hwy - Pearlridge,Company Owned,98-125 Kaonohi Street,Aiea,HI,US,967012318,808-484-9548,GMT-10:00 Pacific/Honolulu,-157.94,21.38,"(21.38, -157.94)"
17205,Starbucks,21005-10034,Pearlridge Mall Uptown,Company Owned,98-1005 Moanalua Road,Aiea,HI,US,967014705,808-484-9355,GMT-10:00 Pacific/Honolulu,-157.94,21.39,"(21.39, -157.94)"
17206,Starbucks,21063-101700,Waimalu Shopping Center,Company Owned,"98-1277 Kaahumanu Street, Building E, Unit 7, ...",Aiea,HI,US,967015314,808-484-5802,GMT-10:00 Pacific/Honolulu,-157.95,21.39,"(21.39, -157.95)"


Calculate the distance matrix using the starter code/function geo_sim_matrix() provided earlier in the notebook. It assumes the column containing the coordinates for each location is called "Coordinates". Examine the docstring for more details.

Note: the latitude and longitudes provided only go out to two decimal places, which limits the resolution of the distance calculations to about 0.5 miles. Distances that are very small may not be accurately represented here (e.g. several instances of "0 distance" for distinct Starbucks locations in very close proximity).

In [8]:
# answer goes here

starbucks_hi_dist = geo_sim_matrix(starbucks_hi_df)
starbucks_hi_dist.head(10)


index,17202,17203,17204,17205,17206,17207,17208,17209,17210,17211,...,17291,17292,17293,17294,17295,17296,17297,17298,17299,17300
index,,,,,,,,,,,,,,,,,,,,,
17202,0.000000,0.690934,0.643386,0.944091,1.460500,6.743576,6.743576,218.428635,218.976944,217.137772,...,109.065346,12.792754,2.373938,16.607107,166.981064,107.264542,98.783943,5.329021,5.675488,6.470637
17203,0.690934,0.000000,0.944121,1.524305,1.888212,6.417289,6.417289,218.067474,218.616730,216.778824,...,108.834190,13.215210,2.050284,16.793444,166.565231,106.962166,98.552231,5.548625,6.120771,6.580589
17204,0.643386,0.944121,0.000000,0.690934,0.944091,6.199704,6.199704,218.976944,219.525787,217.687332,...,109.671234,12.281363,2.921233,15.985024,167.494705,107.842613,99.389603,4.710641,5.179933,5.831356
17205,0.944091,1.524305,0.690934,0.000000,0.643342,6.609524,6.609524,219.338745,219.886642,218.046921,...,109.904938,11.865932,3.304557,15.819399,167.911057,108.146912,99.624100,4.555939,4.747018,5.790081
17206,1.460500,1.888212,0.944091,0.643342,0.000000,6.121677,6.121677,219.886642,220.435089,218.596085,...,110.509941,11.349262,3.826992,15.192343,168.424301,108.724060,100.228790,3.921273,4.240594,5.146738
17207,6.743576,6.417289,6.199704,6.609524,6.121677,0.000000,0.000000,221.634277,222.191803,220.365936,...,113.472672,11.936787,7.995824,12.795587,169.633499,111.089050,103.197449,4.879154,7.028143,4.195248
17208,6.743576,6.417289,6.199704,6.609524,6.121677,0.000000,0.000000,221.634277,222.191803,220.365936,...,113.472672,11.936787,7.995824,12.795587,169.633499,111.089050,103.197449,4.879154,7.028143,4.195248
17209,218.428635,218.067474,218.976944,219.338745,219.886642,221.634277,221.634277,0.000000,0.650494,1.527302,...,113.927498,231.177460,216.060562,234.353195,54.617828,112.120003,123.401138,223.542099,224.074646,224.288895
17210,218.976944,218.616730,219.525787,219.886642,220.435089,222.191803,222.191803,0.650494,0.000000,1.897872,...,114.396652,231.724091,216.609131,234.908646,55.244801,112.634476,123.885117,224.092758,224.622040,224.841476


For each Starbucks, identify its nearest neighboring location in Hawaii (and presumably in the world). Save the output to a dataframe with three columns: Location, Nearest Neighbor, and Distance (Miles).

In [9]:
# answer goes here

nearest = []
for i in starbucks_hi_dist.index:
    dists = starbucks_hi_dist[i]
    dists = dists[dists.index != i]
    nearest.append((i, dists.idxmin(), dists.min()))
    

nearest_df = pd.DataFrame(columns=['Location', 'Nearest Neighbor', 'Distance (Miles)'])
nearest_df['Location'] = [nearest[x][0] for x in range(len(nearest))]
nearest_df['Nearest Neighbor'] = [nearest[x][1] for x in range(len(nearest))]
nearest_df['Distance (Miles)'] = [nearest[x][2] for x in range(len(nearest))]
nearest_df


,Location,Nearest Neighbor,Distance (Miles)
0,17202,17204,0.643386
1,17203,17202,0.690934
2,17204,17202,0.643386
3,17205,17206,0.643342
4,17206,17205,0.643342
...,...,...,...
94,17296,17277,2.838297
95,17297,17255,1.291034
96,17298,17300,1.460423
97,17299,17288,1.524194


If the nearest neighbor of a Starbucks location is far away, we could consider that Starbucks to be "on an island". Which five Starbucks in Hawaii are the most "on an island"?

In [10]:
# answer goes here

statbucks_hi_df2 = nearest_df.merge(starbucks_hi_df[['City','Coordinates']], left_on='Location', right_index=True)
statbucks_hi_df2

,Location,Nearest Neighbor,Distance (Miles),City,Coordinates
0,17202,17204,0.643386,Aiea,"(21.38, -157.93)"
1,17203,17202,0.690934,Aiea,"(21.37, -157.93)"
2,17204,17202,0.643386,Aiea,"(21.38, -157.94)"
3,17205,17206,0.643342,Aiea,"(21.39, -157.94)"
4,17206,17205,0.643342,Aiea,"(21.39, -157.95)"
...,...,...,...,...,...
94,17296,17277,2.838297,Wailea,"(20.69, -156.44)"
95,17297,17255,1.291034,Wailuku,"(20.89, -156.49)"
96,17298,17300,1.460423,Waipahu,"(21.4, -158.01)"
97,17299,17288,1.524194,Waipahu,"(21.43, -158.0)"


In [11]:
statbucks_hi_df2 = statbucks_hi_df2.merge(starbucks_hi_df[['City','Coordinates']],
                left_on='Nearest Neighbor', 
                right_index=True,
                suffixes=(None, ' Nearest')
                )
statbucks_hi_df2.sort_values(by='Distance (Miles)', ascending=False).head()

,Location,Nearest Neighbor,Distance (Miles),City,Coordinates,City Nearest,Coordinates Nearest
93,17295,17267,15.612447,Waikoloa,"(19.91, -155.88)",Kamuela,"(20.02, -155.67)"
65,17267,17295,15.612447,Kamuela,"(20.02, -155.67)",Waikoloa,"(19.91, -155.88)"
76,17278,17284,8.059888,Koloa,"(21.88, -159.46)",Lihue,"(21.97, -159.38)"
89,17291,17261,7.322419,Pukalani,"(20.84, -156.34)",Kahului Airport,"(20.89, -156.44)"
92,17294,17292,6.739833,Waianae,"(21.44, -158.18)",Schofield Barracks,"(21.49, -158.09)"
